In [ ]:
## script: hashing_base.ipynb 
## author: california policy lab 
## purpose: this is a general-purpose hashing script that will hash the 
##          following identifying variables (and various permutations of those variables)
##          in the source data. This script should be modified for each dataset it is run on.
##          1. SSN
##          2. DOB
##             - Day
##             - Month
##             - Year
##          3. Name
##             - First
##             - Last
##      Some commonly needed modifications include:
##      1. Changing the salt to whatever is applicable for the project
##      2. Changing the latest year to the year corresponding to the latest year of data 
##      3. Changing the variable names to correspond to the raw data 
##      4. Modifying as necessary to hash additional PII, or PII across multiple files 


In [ ]:
## import packages 

import pandas as pd
import numpy as np
import datetime
import hashlib
from abydos.phonetic import Soundex
import re
from pathlib import Path

import pdb

In [ ]:
# Show more columns
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_rows', 100)
# Turn on debugger for errors
# %pdb on

In [ ]:
special_characters = pd.read_csv(
    'Unicode Values Sheet - unicode_short (1).csv',
    dtype=str
)
# Clean up the column names a little
special_characters = special_characters.rename(columns={x: x.strip() for x in special_characters.columns})
special_characters

In [ ]:
special_characters_dict = special_characters.set_index('Symbol')['To Value'].to_dict()
special_characters_dict

In [ ]:
def clean_and_preprocess_data(
    input_filepath,
    ssn,
    first_name,
    last_name,
    dob,
    latest_year,
    date_format=None,
    coerce_date=False,
    **kwargs
):
    
    # import raw data, using strings for all columns except DOB, which is a date
    raw = pd.read_csv(
        input_filepath,
        dtype=str,
        parse_dates=[dob] if dob and not date_format else [],
        # Don't want anything to be read as NaN so that the substrings etc will be well defined;
        # we will replace all empty strings with NaN as we gp
        keep_default_na=False,
        **kwargs
    )
    print('CSV imported')
    
    # Read in the table of special characters we want to convert
    special_characters = pd.read_csv(
        'Unicode Values Sheet - unicode_short (1).csv',
        dtype=str
    )
    # Clean up the column names a little
    special_characters = special_characters.rename(columns={x: x.strip() for x in special_characters.columns})
    # Convert to a dictionary
    special_characters_dict = special_characters.set_index('Symbol')['To Value'].to_dict()

    if date_format:
        # Need to parse datetimes after reading in the CSV if it's a custom format; this is the
        # the way pandas recommends you do it in their read_csv documentation
        raw[dob] = pd.to_datetime(raw[dob], format=date_format, errors='coerce' if coerce_date else 'raise')

    if ssn:
        ## clean SSNs
        ## remove all non-digit characters 
        raw['ssn'] = raw[ssn].str.replace(r'[^0-9]', '', regex=True)
        # Pad SSNs to 9 characters
        raw['ssn'] = raw['ssn'].str.pad(width=9, side='left', fillchar='0')
        ## tag invalid SSNs    
        raw['badSSN'] = (
            (raw['ssn'] == '') |
            (raw['ssn'].str.len() > 9) |
            (raw['ssn'].str[0:3].isin(["000", "666"])) |
            (raw['ssn'].str[3:5]=="00") |
            (raw['ssn'].str[5:9]=="0000") |
            (raw['ssn'].str.startswith('9')) |
            (raw['ssn'].isin(["078051120", "123456789"]))
        )
        # Make SSN missing where badSSN is true
        raw['ssn'] = raw['ssn'].mask(raw['badSSN'])

        ## create ssn substrings to hash
        # Do this with a loop over the first of the two digits to hold out
        for i in range(1, 9):
            # ie `raw['ssn56'] = raw['ssn'].str[0:4] + raw['ssn'].str[6:9]
            # Only want to do this where badSSN is not true
            # This implicitly sets NaN where the condition in loc isn't met
            raw.loc[~raw['badSSN'], 'ssn' + str(i) + str(i+1)] = raw['ssn'].str[0:i-1] + raw['ssn'].str[i+1:9]

        
    ## clean names


    pe = Soundex()
    if first_name:
        ## remove common prefixes/suffixes from first & last name, making sure to escape the periods
        # so we can use them in a regex
        fn_prefix_titles = ['mr', 'mrs', 'dr', 'ms']
        fn_prefix_formats = ['{}\\ ', '{}\\.', '{}$']
        # Take cartesian product of the title and possible formats for them and make them a single "or" regex
        fn_prefixes = '|'.join([prefix_format.format(title) for prefix_format in
                       fn_prefix_formats for title in fn_prefix_titles])
        raw['fn'] = (raw[first_name]
                     # str.maketrans makes a translation table from a dictionary; it converts the special characters
                     # to numeric codes, which it can then evidently parse (it doesn't work without the call to maketrans)
                     .str.translate(str.maketrans(special_characters_dict))  # Convert to 26-character alphabet characters
                     .str.lower()  # Make lowercase
                     .str.strip()  # Strip whitespace at ends of string
                     # Replace prefixes at beginning of string
                     .str.replace(r'^({})+'.format(fn_prefixes), '', regex=True)
                     # Remove all non-alphabetical characters
                     .str.replace(r'[^a-z]', '', regex=True))
        # ie the regex this gives will be '^(mr|mrs|dr\.|...|ms\.)', meaning match any of those at the start
        # of the string, and analogously at the end of the string for ln
        # Fill both with nan where empty
        raw['fn'] = raw['fn'].mask(raw['fn'] == '')
        # Tag bad names
        raw['badFN'] = raw['fn'].isnull()
        ## create shortened and soundex vesions of names, for non bad names
        raw.loc[~raw['badFN'], 'fn2l'] = raw.loc[~raw['badFN'], 'fn'].str[0:2]
        raw.loc[~raw['badFN'], 'fn4l'] = raw.loc[~raw['badFN'], 'fn'].str[0:4]
        # Add 1l in here temporarily for comparison to old methodology
        # TODO: delete me
        raw.loc[~raw['badFN'], 'fn1l'] = raw.loc[~raw['badFN'], 'fn'].str[0:1]
        raw.loc[~raw['badFN'], 'fn_sdx'] = raw.loc[~raw['badFN'], 'fn'].apply(pe.encode)

    if last_name:
        ln_suffixes = '|'.join([re.escape(x) for x in [' jr',' sr', ' jr.', ' sr.']])
        raw['ln'] = (raw[last_name]
                     # str.maketrans makes a translation table from a dictionary; it converts the special characters
                     # to numeric codes, which it can then evidently parse (it doesn't work without the call to maketrans)
                     .str.translate(str.maketrans(special_characters_dict))  # Convert to 26-character alphabet characters
                     .str.lower()
                     .str.strip()
                     # Replace suffixes at end of string
                     .str.replace(r'({})+$'.format(ln_suffixes), '', regex=True)
                     .str.replace(r'[^a-z]', '', regex=True))
        raw['ln'] = raw['ln'].mask(raw['ln'] == '')
        raw['badLN'] = raw['ln'].isnull()
        raw.loc[~raw['badLN'], 'ln2l'] = raw.loc[~raw['badLN'], 'ln'].str[0:2]
        raw.loc[~raw['badLN'], 'ln4l'] = raw.loc[~raw['badLN'], 'ln'].str[0:4]
        raw.loc[~raw['badLN'], 'ln1l'] = raw.loc[~raw['badLN'], 'ln'].str[0:1]
        raw.loc[~raw['badLN'], 'ln_sdx'] = raw.loc[~raw['badLN'], 'ln'].apply(pe.encode)

## clean DOB
    ## extract day, month & year
    # Missing values are populated with pd.NaT (Not a Time, pandas' representation of missing dates)
    # when parsing the csv, even if keep_default_na is set to False (which makes sense, not sure what
    # else they could be)
    # NaT.year gives NaN, so this propagates missing values appropriately
    if dob:
        raw['dob_y'] = raw[dob].dt.year
        raw['dob_m'] = raw[dob].dt.month
        raw['dob_d'] = raw[dob].dt.day
        ## tag invalid dates
        raw['badDOB'] = (
            (pd.isnull(raw[dob])) |
            (raw['dob_y'] < 1900) |
            (raw['dob_y'] > latest_year)
        )
        ## convert to string for hashing to work (where we haven't flagged it as bad)
        raw.loc[~raw['badDOB'], 'dob_y'] = raw.loc[~raw['badDOB'], 'dob_y'].apply(lambda x: '{:.0f}'.format(x))
        raw.loc[~raw['badDOB'], 'dob_m'] = raw.loc[~raw['badDOB'], 'dob_m'].apply(lambda x: '{:.0f}'.format(x))
        raw.loc[~raw['badDOB'], 'dob_d'] = raw.loc[~raw['badDOB'], 'dob_d'].apply(lambda x: '{:.0f}'.format(x))

    return raw


## main function to import, clean, and hash data 
def hash_data(salt, 
              input_filepath, 
              output_filepath,  
              ssn, 
              first_name, 
              last_name, 
              dob,
              latest_year,
              drop_unhashed_pii=True,
              coerce_date=False,
             **kwargs):
    """ Hash PII (SSN, first name, last name, and date of birthday) and derivatives
        from an input CSV file. 
        
        Parameters 
        ----------
        
        salt: string 
            the agreed-upon salt  
        input_filepath: string 
            the local filepath of the input csv 
        output_filepath: string
            the local filepath of the output csv 
        ssn: string
            the name of the ssn variable in the input csv 
        first_name: string 
            the name of the first name variable in the input csv 
        last_name: string 
            the name of the last name variable in the input csv 
        dob: string
            the name of the birthdate variable in the input csv
        latest_year: integer 
            the latest plausible birthyear in the input data
        kwargs: dict
            Keyword arguments that are passed to clean_and_preprocess_data
       
        Returns
        -------
        
        hashed data: a csv file with all programmatic data plus hashed PII 
    """

    raw = clean_and_preprocess_data(
        input_filepath=input_filepath,
        ssn=ssn,
        first_name=first_name,
        last_name=last_name,
        dob=dob,
        latest_year=latest_year,
        coerce_date=coerce_date,
        **kwargs
    )
    
    ## drop intermediate & raw vars, do some renaming    
    vars_to_hash = (
        (['ssn','ssn89','ssn78','ssn67','ssn56','ssn45',
         'ssn34','ssn23','ssn12'] if ssn else []) +
        (['fn', 'fn2l', 'fn1l', 'fn4l','fn_sdx'] if first_name else []) +
        (['ln', 'ln2l', 'ln1l', 'ln4l','ln_sdx'] if last_name else []) +
        (['dob_d', 'dob_m', 'dob_y'] if dob else [])
)
    clean = raw[vars_to_hash + [x for x in raw.columns if x not in vars_to_hash and 
                                x not in ([ssn, first_name, last_name, dob] if drop_unhashed_pii else [])]]
    

## hash SSN and derivatives, first and last name and derivatives, and DOB and derivatives
#     pdb.set_trace()
    for col in vars_to_hash:
#         print(col)
        # Get which base field this is a part of, so we can not hash fields where it's flagged as bad
        this_base_col = re.search(r'^([a-z]+)', col).group(1)
        this_bad_flag = 'bad' + this_base_col.upper()
        clean.loc[~clean[this_bad_flag], col + '_hash'] = clean.loc[~clean[this_bad_flag], col].apply(
            lambda s: hashlib.sha256(str.encode(salt+s)).hexdigest().upper()
        )

## drop unhashed PII and export
    if drop_unhashed_pii:
        clean = clean.drop(columns=vars_to_hash)
    clean.to_csv(output_filepath, sep=",", index=False)  
    
    return clean

## DOB

In [ ]:
hashed = hash_data(
    salt="FAKE2!59U~9668SALT",
    input_filepath="", 
    output_filepath="", 
    ssn=None, 
    first_name=None, 
    last_name=None, 
    dob='',
    latest_year=2021,
    sep=',',
    drop_unhashed_pii=False,
    coerce_date=True,
    date_format='%Y-%m-%d'
)
hashed

## SSN

In [ ]:
hashed = hash_data(
    salt="FAKE2!59U~9668SALT",
    input_filepath="", 
    output_filepath="", 
    ssn='', 
    first_name=None, 
    last_name=None, 
    dob=None,
    latest_year=2021,
    sep='|',
    drop_unhashed_pii=False
)
hashed

## FN/LN

In [ ]:
import csv
hashed = hash_data(
    salt="FAKE2!59U~9668SALT",
    input_filepath="", 
    output_filepath="", 
    ssn=None, 
    first_name='', 
    last_name='',
    dob=None,
    latest_year=2021,
    sep='|',
    nrows=None,
    drop_unhashed_pii=False,
    error_bad_lines=False,
    warn_bad_lines=True,
    quoting=csv.QUOTE_NONE
)
hashed